In [1]:
import os
import numpy as np
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

## Load Data

In [2]:
path = 'TESTDATA'
actions = np.array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'check', 'finish', 'give_you', 'good', 'i', 'id_card', 'is', 'money', 'saving_book', 'sign', 'taiwan', 'take', 'ten_thousand', 'yes'])
sequence = 115
total = -1
temp = np.ones([2760,30,258])

for i in range(1,len(actions)+1):
    ACTION_PATH = os.path.join(path, actions[i-1])
    for j in range(1,sequence+1):
        total += 1
        DATA_PATH = os.path.join(ACTION_PATH, str(j-1))
        for k in range(1,30+1):
            real_path = (os.path.join(DATA_PATH, str(k-1)+'.npy'))
            real_data = np.load(real_path, allow_pickle=True)
            temp[total][k-1] = real_data

np.save('data-merge-0713.npy', temp)

FileNotFoundError: [Errno 2] No such file or directory: 'TESTDATA/0/0/0.npy'

In [67]:
# 1. 打包模組需要的 Dataset
sequences = np.load(os.path.join('data-merge-0713.npy'))
headsSequences = sequences[:, :, 132:258] #132:258
headsSequences.shape

(3105, 30, 126)

In [123]:
# 2. 模組需要的字詞 Labels
actions = np.array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'check', 'finish', 'give_you', 'good', 'i', 'id_card', 'is', 'money', 'saving_book', 'sign', 'taiwan', 'take', 'ten_thousand', 'yes','deposit','get','job'])


label_map = {label:num for num, label in enumerate(actions)}
print(label_map)
print(actions.shape[0])

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'check': 10, 'finish': 11, 'give_you': 12, 'good': 13, 'i': 14, 'id_card': 15, 'is': 16, 'money': 17, 'saving_book': 18, 'sign': 19, 'taiwan': 20, 'take': 21, 'ten_thousand': 22, 'yes': 23, 'deposit': 24, 'get': 25, 'job': 26}
27


In [69]:
# 115 是每個字詞的資料量，可以再修改
labels = []
for action in actions:
    for _ in range(115):
        labels.append(label_map[action])
print([[actions[l], l] for l in labels[::115]])

[['0', 0], ['1', 1], ['2', 2], ['3', 3], ['4', 4], ['5', 5], ['6', 6], ['7', 7], ['8', 8], ['9', 9], ['check', 10], ['finish', 11], ['give_you', 12], ['good', 13], ['i', 14], ['id_card', 15], ['is', 16], ['money', 17], ['saving_book', 18], ['sign', 19], ['taiwan', 20], ['take', 21], ['ten_thousand', 22], ['yes', 23], ['job', 24], ['deposit', 25], ['get', 26]]


In [70]:
# Reshape (-1,30,258) 是讓所有的資料變成z:258, y:30 的array
X = headsSequences.reshape(-1,30,126) #258
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(3105, 30, 126)
(2794, 30, 126)
(311, 30, 126)


# Train model 

In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

In [74]:
model = Sequential()
model.add(GRU(128, activation='relu', input_shape=(30,126),return_sequences=True)) # LSTM -> GRU
model.add(GRU(64, activation='relu', input_shape=(30,126)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_12 (GRU)                (None, 30, 128)           98304     
                                                                 
 gru_13 (GRU)                (None, 64)                37248     
                                                                 
 dense_14 (Dense)            (None, 64)                4160      
                                                                 
 dense_15 (Dense)            (None, 32)                2080      
                                                                 
 dense_16 (Dense)            (None, 27)                891       
                                                                 
Total params: 142683 (557.36 KB)
Trainable params: 142683 (557.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [75]:
callback = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, epochs=50, callbacks=[callback])

Epoch 1/50
88/88 [==============================] - 3s 17ms/step - loss: 3.0108 - categorical_accuracy: 0.1067
Epoch 2/50
88/88 [==============================] - 2s 19ms/step - loss: 2.5642 - categorical_accuracy: 0.1858
Epoch 3/50
88/88 [==============================] - 2s 19ms/step - loss: 2.2449 - categorical_accuracy: 0.2212
Epoch 4/50
88/88 [==============================] - 2s 20ms/step - loss: 2.0643 - categorical_accuracy: 0.2709
Epoch 5/50
88/88 [==============================] - 2s 19ms/step - loss: 1.8813 - categorical_accuracy: 0.3228
Epoch 6/50
88/88 [==============================] - 2s 19ms/step - loss: 1.7703 - categorical_accuracy: 0.3558
Epoch 7/50
88/88 [==============================] - 2s 19ms/step - loss: 1.5323 - categorical_accuracy: 0.4424
Epoch 8/50
88/88 [==============================] - 2s 19ms/step - loss: 1.3636 - categorical_accuracy: 0.5122
Epoch 9/50
88/88 [==============================] - 2s 19ms/step - loss: 1.2258 - categorical_accuracy: 0.5329
E

In [76]:
def accuracy(X, y):
    print(X.shape, y.shape)
    res = model.predict(X)
    accuracy = (np.argmax(res, axis=1) == np.argmax(y, axis=1)).sum()/len(res)
    return accuracy

In [77]:
print(accuracy(X_train, y_train))
print(accuracy(X_test, y_test))

(2794, 30, 126) (2794, 27)
88/88 [==============================] - 1s 5ms/step
0.9280601288475304
(311, 30, 126) (311, 27)
10/10 [==============================] - 0s 5ms/step
0.9131832797427653


# Save & Load Model

In [79]:
model.save("Model/GRU")

INFO:tensorflow:Assets written to: Model/GRU/assets


INFO:tensorflow:Assets written to: Model/GRU/assets


In [80]:
# model.save("Model/GRU")
model.save('model_0730.keras')

In [81]:
model.save('model_0730.h5')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [82]:
from tensorflow.keras.models import load_model

In [83]:
new_model = load_model("Model/GRU")
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_12 (GRU)                (None, 30, 128)           98304     
                                                                 
 gru_13 (GRU)                (None, 64)                37248     
                                                                 
 dense_14 (Dense)            (None, 64)                4160      
                                                                 
 dense_15 (Dense)            (None, 32)                2080      
                                                                 
 dense_16 (Dense)            (None, 27)                891       
                                                                 
Total params: 142683 (557.36 KB)
Trainable params: 142683 (557.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [84]:
new_model = load_model("model_0730.h5")
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_12 (GRU)                (None, 30, 128)           98304     
                                                                 
 gru_13 (GRU)                (None, 64)                37248     
                                                                 
 dense_14 (Dense)            (None, 64)                4160      
                                                                 
 dense_15 (Dense)            (None, 32)                2080      
                                                                 
 dense_16 (Dense)            (None, 27)                891       
                                                                 
Total params: 142683 (557.36 KB)
Trainable params: 142683 (557.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [85]:
print(accuracy(X_train, y_train))

(2794, 30, 126) (2794, 27)
88/88 [==============================] - 0s 5ms/step
0.9280601288475304


In [86]:
res = new_model.predict(X_test)
accuracy = (np.argmax(res, axis=1) == np.argmax(y_test, axis=1)).sum()/len(res)
print(accuracy)

10/10 [==============================] - 0s 5ms/step
0.9131832797427653


# TFlite

In [17]:
# import tensorflow as tf

In [18]:
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter._experimental_lower_tensor_list_ops = False
# tflite_model = converter.convert()

# with open('my_model2.tflite', 'wb') as f:
#     f.write(tflite_model)

# Realtime Test

In [135]:
import cv2
import mediapipe as mp

In [136]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [144]:
colors = [(245,117,16)] * 24
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        # cv2.rectangle(影像, 頂點座標, 對向頂點座標, 顏色, 線條寬度)
        cv2.rectangle(output_frame, (0,60+num*17), (int(prob*100), 90+num*17), colors[num], -1) #17,17
        cv2.putText(output_frame, actions[num], (0, 85+num*17), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return output_frame

In [138]:
def mediapipe_detection(image, model):
    # Transfer image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    # Make prediction
    results = model.process(image)
    return results

In [139]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
        mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    )
    # Draw left hand connections
    mp_drawing.draw_landmarks(
        image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
        mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
        mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
    ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(
        image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
    ) 

In [140]:
def extract_keypoints_without_face(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh]) 

In [141]:
def get_dataset(data_path_trans):
    input_texts = []
    target_texts = []

    input_characters = set()
    target_characters = set()
    with open(data_path_trans, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    for line in lines:
        input_text, target_text= line.split('   ')
        # 用tab作用序列的开始，用\n作为序列的结束
        target_text = '\t' + target_text + '\n'

        input_texts.append(input_text)
        target_texts.append(target_text)
        
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)
    return input_texts,target_texts,input_characters,target_characters


#------------------------------------------#
#   init初始化部分
#------------------------------------------#

# 获取数据集
# input_texts為输入的英文手語序 target_texts為對應的中文口語序
# input_characters用到的所有输入字符,如a,b,c,d,e,……,.,!等
data_path_trans = 'EngToChinese.txt'
input_texts,target_texts,input_characters,target_characters = get_dataset(data_path_trans)

# 对字符进行排序
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

# 计算共用到了什么字符
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# 计算出最长的序列是多长
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

# 建立字母到数字的映射
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])
# 求數字到字母的映射
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())
    

model_trans = load_model('model_0730.h5')

In [142]:
def translate(model_opt):
    # model_opt = 'check yes paper sign'
    in_encoder = np.zeros((1, max_encoder_seq_length, num_encoder_tokens),dtype='float32')

    for t, char in enumerate(model_opt):
        in_encoder[0, t, input_token_index[char]] = 1.
    in_encoder[0, t + 1:, input_token_index[' ']] = 1.

    in_decoder = np.zeros((len(in_encoder), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
    in_decoder[:, 0, target_token_index["\t"]] = 1

    # 生成 decoder 的 output
    for i in range(max_decoder_seq_length - 1):
        predict = model_trans.predict([in_encoder, in_decoder])
        predict = predict.argmax(axis=-1)
        predict_ = predict[:, i].ravel().tolist()
        for j, x in enumerate(predict_):
            in_decoder[j, i + 1, x] = 1 # 將每個預測出的 token 設為 decoder 下一個 timestsmp 的輸入

    seq_index = 0
    decoded_sentence = ""
    output_seq = predict[seq_index, :].ravel().tolist()
    for x in output_seq:
        if reverse_target_char_index[x] == "\n":
            break
        else:
            decoded_sentence+=reverse_target_char_index[x]

    # print('Input sentence:', model_opt)
    # print('Decoded sentence:', decoded_sentence)
    return decoded_sentence

In [143]:
# 1. New detection variables
sequence = []
sentence = []

predictions = []
threshold = 0.7
alarm_set = False
trans_result =""

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Make detections
        results = mediapipe_detection(frame, holistic)
        # Draw landmarks
        draw_styled_landmarks(frame, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints_without_face(results)
        if np.count_nonzero(keypoints) > 30:
            sequence.append(keypoints)
            sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = new_model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                            last_updated_time = time.time()
                            alarm_set = True
                    else:
                        sentence.append(actions[np.argmax(res)])
                        last_updated_time = time.time()
                        alarm_set = True

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            frame = prob_viz(res, actions, frame, colors)

        current_time = time.time()  
        if alarm_set and current_time - last_updated_time >= 10:
            # 時間過10秒，將 sentence 放入下一個模型進行預測
            trans_result = translate(' '.join(sentence))
            print('---result---', trans_result)
            # 清空 sentence 資料
            alarm_set = False
            sequence = []
            sentence = []
            
        cv2.rectangle(frame, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(frame, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, trans_result, (300,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', frame)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 9ms/step
1


IndexError: list index out of range

## Testing


In [117]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Make detections
        results = mediapipe_detection(frame, holistic)
        # Draw landmarks
        draw_styled_landmarks(frame, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints_without_face(results)
        if np.count_nonzero(keypoints) > 150:
            sequence.append(keypoints)
            sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = new_model.predict(np.expand_dims(sequence, axis=0))[0]
            # prediction = actions[np.argmax(res)]
            # print(prediction)
            # print(res[np.argmax(res)])
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
                        print(sentence)

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            frame = prob_viz(res, actions, frame, colors)
            
        cv2.rectangle(frame, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(frame, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', frame)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 